In [1]:
import jieba
import json
import pandas as pd
import os
import numpy as np
import csv
import dfply
import re


In [48]:
df = pd.read_excel('E:/tensorflow/crawler/ELLEblog_1.xlsx',encoding='utf-8',index_col=0)
df


,標題,日期,作者,網址,內文
編號,,,,,
0,火辣女神拐拐變身「阿凡達」，僅黏胸貼、穿丁字褲！揭露7小時變裝過程，同步公開健身菜單,2019/11/01,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g296629...,過完萬聖節，不少名人皆在社群上分享自己當日的造型，小丑、安娜貝爾皆為熱門選項，而在網路上擁有...
1,蘭蔻絕對完美唇膏爆紅色號務必抄下！《鬼怪》金高恩#264、秀智專屬色S#360...每支都美到炸,2019/10/30,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g27922/...,這部人氣王《鬼怪》荷爾蒙裂屏的孔侑、單眼皮美女金高恩、高挑有型的李棟旭…無一不吸引著妳，也覺...
2,優雅的大人如何重拾玩心？今年用全新蛻變登場的LUNASOL霓光系列征服五光十色的派對季節,2019/10/28,CHIA CHIU,https://www.elle.com/tw/beauty/make-up/a296039...,喜歡畫大地色系妝容的人，一定都用過有「大地色女王」之美名的LUNASOL，大地色系如此優雅迷...
3,【老闆不說的事】業界最美公關！資生堂傳媒執行長張凡恬：「帶人要帶心！員工有怨氣先帶去喝一杯」,2019/10/24,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g295693...,談到最愛彩妝、保養，所有台灣女生的化妝桌上絕對少不了SHISEIDO資生堂集團的產品！蘿拉蜜...
4,推薦給不張狂、低調優雅的妳！爆紅【可可裸棕】唇膏，6款人氣色號完整試色,2019/10/23,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g244652...,唇彩大勢繼乾燥玫瑰色系、鐵鏽紅色系後，爆紅的「南瓜土橘色」重磅登場！混合暖色調的土色，宛如一...
...,...,...,...,...,...
1161,10位最美金球獎女星征服紅毯全紀錄,2012/01/18,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2596/1...,紅唇搖滾全場-安潔莉娜裘莉安潔莉娜裘莉(AngelinaJolie)與黛安娜艾格倫(Dian...
1162,2012好萊塢美女預測新亮相,2012/01/12,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2595/2...,FelicityJones大家可能對這位英國女演員不太熟悉(演過TheWorstWitch和...
1163,好想咬一口！打造女星們的水蜜桃美肌,2012/01/10,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2594/c...,亞曼達賽佛瑞(AmandaSeyfried)亞曼達賽佛瑞(AmandaSeyfried)天生...


In [14]:
df = df.drop(columns = ['Unnamed: 0'])
df

,標題,日期,作者,網址,內文
0,火辣女神拐拐變身「阿凡達」，僅黏胸貼、穿丁字褲！揭露7小時變裝過程，同步公開健身菜單,2019/11/01,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g296629...,過完萬聖節，不少名人皆在社群上分享自己當日的造型，小丑、安娜貝爾皆為熱門選項，而在網路上擁有...
1,蘭蔻絕對完美唇膏爆紅色號務必抄下！《鬼怪》金高恩#264、秀智專屬色S#360...每支都美到炸,2019/10/30,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g27922/...,這部人氣王《鬼怪》荷爾蒙裂屏的孔侑、單眼皮美女金高恩、高挑有型的李棟旭…無一不吸引著妳，也覺...
2,優雅的大人如何重拾玩心？今年用全新蛻變登場的LUNASOL霓光系列征服五光十色的派對季節,2019/10/28,CHIA CHIU,https://www.elle.com/tw/beauty/make-up/a296039...,喜歡畫大地色系妝容的人，一定都用過有「大地色女王」之美名的LUNASOL，大地色系如此優雅迷...
3,【老闆不說的事】業界最美公關！資生堂傳媒執行長張凡恬：「帶人要帶心！員工有怨氣先帶去喝一杯」,2019/10/24,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g295693...,談到最愛彩妝、保養，所有台灣女生的化妝桌上絕對少不了SHISEIDO資生堂集團的產品！蘿拉蜜...
4,推薦給不張狂、低調優雅的妳！爆紅【可可裸棕】唇膏，6款人氣色號完整試色,2019/10/23,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g244652...,唇彩大勢繼乾燥玫瑰色系、鐵鏽紅色系後，爆紅的「南瓜土橘色」重磅登場！混合暖色調的土色，宛如一...
...,...,...,...,...,...
1161,10位最美金球獎女星征服紅毯全紀錄,2012/01/18,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2596/1...,紅唇搖滾全場-安潔莉娜裘莉安潔莉娜裘莉(AngelinaJolie)與黛安娜艾格倫(Dian...
1162,2012好萊塢美女預測新亮相,2012/01/12,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2595/2...,FelicityJones大家可能對這位英國女演員不太熟悉(演過TheWorstWitch和...
1163,好想咬一口！打造女星們的水蜜桃美肌,2012/01/10,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2594/c...,亞曼達賽佛瑞(AmandaSeyfried)亞曼達賽佛瑞(AmandaSeyfried)天生...
1164,【彩妝放大鏡】名人姐妹妝容大不同,2012/01/04,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2593/c...,雖然是雙胞胎，歐森姊妹對於品味卻有著顯著的差距，可以輕易分辨兩人。姊姊Ashley喜歡混搭風...


In [92]:
df['編號']

KeyError: '編號'

In [121]:
df = pd.read_excel('E:/tensorflow/textMinning/elle_teach/elle_face_teach.xlsx',encoding='utf-8',index_col=None)


In [151]:
#找出關鍵字合併
# print(str(df))
# jieba.load_userdict("aaa.txt")
# temp = jieba.cut(str(df))
temp_all = 0
# df["Indexes"] = df["內文"].str.find('遮瑕')
df["Indexes1"] = df["內文"].str.find('粉底')
df["Indexes2"] = df["內文"].str.find('粉底液')
df["Indexes3"] = df["內文"].str.find('底妝')
df["Indexes4"] = df["內文"].str.find('裸妝')
df["Indexes5"] = df["內文"].str.find('光圈')

# temp = df[df["Indexes"] != -1]
temp1 = df[df["Indexes1"] != -1]
temp2 = df[df["Indexes2"] != -1]
temp3 = df[df["Indexes3"] != -1]
temp4 = df[df["Indexes4"] != -1]
temp5 = df[df["Indexes5"] != -1]

temp_all = pd.concat([temp1,temp2,temp3,temp4,temp5], axis=0,ignore_index=True)
temp_all_1= temp_all.drop_duplicates(subset=None, keep='first')




In [20]:
df["Indexes"] = df["標題"].str.find('眼皮')
df["Indexes1"] = df["內文"].str.find('眼圈')
df["Indexes3"] = df["內文"].str.find('眼妝')
df["Indexes4"] = df["內文"].str.find('眼影')
df["Indexes5"] = df["內文"].str.find('眼線')


temp = df[df["Indexes"] != -1]
temp_num = temp['Unnamed: 0']

temp1 = df[df["Indexes1"] != -1]
temp1
# temp_num_1 = temp['Unnamed: 0']

# temp2 = df[df["Indexes3"] != -1]
# temp_num_2 = temp['Unnamed: 0']

df1=df.drop(temp_num)
# df=df.drop(temp_num_1)
# df=df.drop(temp_num_2)
df1
# temp3 = df[df["Indexes3"] != -1]
# temp4 = df[df["Indexes4"] != -1]
# temp5 = df[df["Indexes5"] != -1]


,Unnamed: 0,Unnamed: 0.1,標題,日期,作者,網址,內文,Indexes,Indexes1,Indexes2,Indexes3,Indexes4,Indexes5
0,0,0,殘酷的年齡殺～「法令紋」遮瑕大法學起來！這關鍵性的小技巧真心實用啊,2019/10/17,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g294675...,皺紋是女人的頭號敵人，尤其是法令紋容易讓人看起來嚴肅、疲累、顯老！除了勤擦保養品以外，簡單的...,-1,-1,297,-1,-1,-1
1,1,1,PTT討論度最高！8款網紅「粉底液」真人實測評比～混和油肌到底哪款適合？完整解析,2019/09/26,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g292383...,粉底控注意！底妝是絕美妝感的先決條件，用到不適合自己的粉底液，肌膚質感絕對比素顏還不堪入目～...,-1,-1,27,-1,-1,-1
2,2,2,美妝蛋、多角、波浪形底妝海綿怎麼挑？人氣彩妝師王盈喬還有「消水腫」用法⋯超強訣竅必須記,2019/09/10,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g282467...,美妝蛋、多角海綿怎麼選？昂貴的粉撲值得買嗎？這是許多人在逛藥妝店時經常出現的疑問。這次ELL...,-1,-1,-1,-1,-1,-1
3,3,3,2.5秒完成C位光芒的柔霧光底妝！精品氣墊女王DIOR「超完美底妝CP」24小時不脫妝教學全...,2019/08/22,KAHLEN CHEN,https://www.elle.com/tw/beauty/make-up/g286941...,夏季上妝時，女孩們最怕高溫炙熱引發土石流，修修補補後反而更慘不忍睹…為此，研究如何讓妝感一整...,-1,690,151,-1,-1,658
4,4,4,2019霸氣「黑色氣墊」粉餅推薦！控油持妝、保濕遮瑕...PTT討論爆棚完整評比,2019/08/21,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g287688...,今年，底妝市場仍然是群雄爭戰！DIOR、M.A.C、NARS、YSL、植村秀和資生堂氣墊粉餅...,-1,1176,818,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,376,376,碧兒泉可調色的隔離霜,2012/03/01,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/a10470/...,精品講求客製化、獨一無二，那肌膚呢？率先提出「可調色」隔離防護概念的碧兒泉，就像妳專屬的超級...,-1,-1,-1,-1,-1,-1
325,377,377,【彩妝心理學】從你的妝容看個性,2012/02/13,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2604/w...,煙薰妝時常可以展現女性神秘的一面，並讓人感覺到這是一位「有著極佳的品味與聰穎自信」的女性。彩...,-1,-1,-1,134,485,-1
326,378,378,花邊教主初長成主角彩妝大變身,2012/01/20,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2598/g...,Serena初登場第一季的Serena，剛回到上東區，身穿高中制服，一頭亞麻色微捲的髮型在當...,-1,-1,-1,65,-1,-1
327,379,379,【彩妝放大鏡】名人姐妹妝容大不同,2012/01/04,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2593/c...,雖然是雙胞胎，歐森姊妹對於品味卻有著顯著的差距，可以輕易分辨兩人。姊姊Ashley喜歡混搭風...,-1,-1,-1,127,256,77


In [18]:
df1.to_json('./elle_teach/elle_face_teach.json')
df1.to_excel('./elle_teach/elle_face_teach.xlsx')

In [122]:
df.to_csv('./elle_teach/elle_face_teach.csv')

In [113]:
df = pd.read_excel('E:/tensorflow/textMinning/elle_teach/elle_eye1_teach.xlsx',encoding='utf-8',index_col=0)
df

,Unnamed: 0.1,標題,日期,作者,網址,內文,Indexes,Indexes1,Indexes2,Indexes3,Indexes4,Indexes5
1,1,【TOMFORD】三款秋冬強運眉型示範教學！,2019/09/01,YVONNE,https://www.elle.com/tw/beauty/make-up/g288780...,TOMFORD堅持每個人都是獨一無二的，也因此，該以專屬的方式設計最適合你的美麗！他將高級訂...,54,-1,-1,1419,-1,-1
2,2,「原來眉毛沒有型，氣質就很難突顯出來啊！」找對眉型臉整個小一號，超簡單圖解看完秒懂,2019/05/16,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g274888...,是不是覺得妝怎麼畫，就是少了一股空靈、氣質感？其實眉毛就是很重要的關鍵！不只是將空隙填滿，其...,25,-1,-1,-1,990,405
3,3,「只是改變眉頭畫法，就被誇變漂亮！」眉型的關鍵就在這裡，超強圖解步驟一看就秒懂,2019/05/14,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g274639...,眉毛真的是一個很神奇的部位，稍為偏一點、往前或高一點，整張臉的感覺就不一樣！可以說影響整體妝...,0,-1,-1,-1,-1,-1
4,4,「瞬間讓大小眼對稱、雙眼消浮腫」絕對要學好，你還不知道【大地棕色眼影】超級神用法,2019/01/30,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g260816...,無論哪個年紀，你都需要一輩子都能派得上用場的妝容！日本最暢銷的化妝書《完妝後就是美人》將天后...,298,-1,99,564,198,366
5,5,國際彩妝大師張景凱親授！搭配韓國研發新品精準描繪，連彩妝新手也＂眉＂問題！,2019/01/19,YVONNE,https://www.elle.com/tw/beauty/make-up/g259584...,為了讓缺乏專業彩妝技巧的素人們，也都能夠輕鬆畫出以假亂真的心機眉型，隨心所欲創造想要的搭襯眉...,137,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
276,276,「遮瑕選對了，人生更完整啊！」NARS、GA、M.A.C…12款神級遮瑕筆實測評比，一抹忘了...,2019/08/22,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g264223...,黑眼圈、眼下細紋都所有女人上妝時最大困擾，市面上遮黑眼圈的產品更是容易踩雷，遮瑕效果佳的，質...,-1,-1,-1,-1,-1,4630
277,277,【游絲棋專欄】不厚重又持妝的「45度睫毛+菱形上妝法」度假不怕太陽曬！！,2016/06/11,NICOLE HUANG,https://www.elle.com/tw/beauty/make-up/g24999/...,出門前三步驟：保養→防曬→上妝就算看起來是陰天，一樣潛藏著把妳曬黑曬老的紫外線！更何況是像我...,-1,-1,-1,-1,-1,706
278,278,眼睛變大一倍的訣竅！基本款貓眼線簡單上手6步驟,2014/08/21,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2436/m...,先在眼尾部分延著上眼線往上勾起，延伸出去。「按照自己的眼型，延伸出去的線條往太陽穴的方向勾起...,-1,-1,-1,-1,-1,9
279,279,米蘭達可兒御用彩妝師開示教你畫出好感度乾淨眼影,2014/06/11,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2407/v...,>>太實用！你還不知道關於眼線的7個小知識>>夏日出遊不出油！嚴防脫妝大作戰,-1,-1,-1,-1,-1,13


In [49]:
temp_all = 0
df["Indexes"] = df["內文"].str.find('眉毛')
df["Indexes1"] = df["內文"].str.find('眉筆')
df["Indexes2"] = df["內文"].str.find('眼睛')
df["Indexes3"] = df["內文"].str.find('眼妝')
df["Indexes4"] = df["內文"].str.find('眼影')
df["Indexes5"] = df["內文"].str.find('眼線')

temp = df[df["Indexes"] != -1]
temp1 = df[df["Indexes1"] != -1]
temp2 = df[df["Indexes2"] != -1]
temp3 = df[df["Indexes3"] != -1]
temp4 = df[df["Indexes4"] != -1]
temp5 = df[df["Indexes5"] != -1]

temp_all = pd.concat([temp,temp1,temp2,temp3,temp4,temp5], axis=0,ignore_index=True)
temp_all_1= temp_all.drop_duplicates(subset=None, keep='first')
temp_all_1.to_excel('./elle_teach/elle_eye1_teach.xlsx')



In [110]:
df["Indexes1"] = df["標題"].str.find('進化')
temp = df[df["Indexes1"] != -1]
temp_num = temp['Unnamed: 0.1']
df1=df.drop(temp_num)
df1

,Unnamed: 0.1,標題,日期,作者,網址,內文,Indexes,Indexes1,Indexes2,Indexes3,Indexes4,Indexes5
1,1,【TOMFORD】三款秋冬強運眉型示範教學！,2019/09/01,YVONNE,https://www.elle.com/tw/beauty/make-up/g288780...,TOMFORD堅持每個人都是獨一無二的，也因此，該以專屬的方式設計最適合你的美麗！他將高級訂...,54,-1,-1,1419,-1,-1
3,3,「原來眉毛沒有型，氣質就很難突顯出來啊！」找對眉型臉整個小一號，超簡單圖解看完秒懂,2019/05/16,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g274888...,是不是覺得妝怎麼畫，就是少了一股空靈、氣質感？其實眉毛就是很重要的關鍵！不只是將空隙填滿，其...,25,-1,-1,-1,990,405
4,4,「只是改變眉頭畫法，就被誇變漂亮！」眉型的關鍵就在這裡，超強圖解步驟一看就秒懂,2019/05/14,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g274639...,眉毛真的是一個很神奇的部位，稍為偏一點、往前或高一點，整張臉的感覺就不一樣！可以說影響整體妝...,0,-1,-1,-1,-1,-1
7,7,「瞬間讓大小眼對稱、雙眼消浮腫」絕對要學好，你還不知道【大地棕色眼影】超級神用法,2019/01/30,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g260816...,無論哪個年紀，你都需要一輩子都能派得上用場的妝容！日本最暢銷的化妝書《完妝後就是美人》將天后...,298,-1,99,564,198,366
8,8,國際彩妝大師張景凱親授！搭配韓國研發新品精準描繪，連彩妝新手也＂眉＂問題！,2019/01/19,YVONNE,https://www.elle.com/tw/beauty/make-up/g259584...,為了讓缺乏專業彩妝技巧的素人們，也都能夠輕鬆畫出以假亂真的心機眉型，隨心所欲創造想要的搭襯眉...,137,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
595,595,米蘭達可兒御用彩妝師開示教你畫出好感度乾淨眼影,2014/06/11,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2407/v...,>>太實用！你還不知道關於眼線的7個小知識>>夏日出遊不出油！嚴防脫妝大作戰,-1,-1,-1,-1,-1,13
596,596,亞洲彩妝大師Zing教妳看眼型畫眼線,2014/06/04,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2404/z...,1.用筆尖從眼頭到眼尾描繪眼線。2.再用筆頭的斜角於眼角處補黑。3.最後再將上眼線向上拉提至...,-1,-1,-1,-1,-1,13
600,600,婚宴妝髮8造型向女星學習優雅之道,2013/04/18,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2285/b...,莎拉潔西卡派克（SarahJessicaParker）的頭髮非常休閒隨性，但是全框起來的眼線...,-1,-1,-1,-1,-1,44
603,603,凱特摩斯無與倫比超模風範,2012/03/05,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2617/k...,Q1：這款唇膏有一些較少見的顏色，像是貴氣十足的皇家紫。妳是從哪裡獲得靈感的呢？Kate：我...,-1,-1,-1,-1,-1,272


In [115]:
df1.to_json('./elle_teach/elle_eye1_teach.json')
df1.to_excel('./elle_teach/elle_eye1_teach.xlsx')


In [120]:
df1.to_csv('./elle_teach/elle_eye1_teach.csv')

In [123]:
df = pd.read_excel('E:/tensorflow/textMinning/elle_teach/elle_lips_teach.xlsx',encoding='utf-8',index_col=None)
df

,Unnamed: 0,Unnamed: 0.1,標題,日期,作者,網址,內文,Indexes,Indexes1,Indexes2,Indexes3,Indexes4,Indexes5
0,0,0,2019最強開架唇膏評比！INTEGRATE、KATE、CLIO、ETUDE⋯不超過500元...,2019/10/12,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g258827...,雖然說專櫃唇膏總是讓人走心，但是近兩年不論是日系、韓系甚至於台灣本土開架彩妝品牌都不停推出適...,1680,-1,-1,52,-1,-1
1,1,1,草莓冰沙冰涼透潤紅美呆！「ARMANI#508、DIOR#976…」小紅書超人氣水潤唇膏推薦,2019/05/30,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g276247...,炎熱的夏季，除了靠穿搭營造換季成清涼配色之外，小紅書上也出現相當多適合夏季的水潤唇膏推薦，尤...,490,-1,-1,161,815,-1
2,2,2,「原來眉毛沒有型，氣質就很難突顯出來啊！」找對眉型臉整個小一號，超簡單圖解看完秒懂,2019/05/16,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g274888...,是不是覺得妝怎麼畫，就是少了一股空靈、氣質感？其實眉毛就是很重要的關鍵！不只是將空隙填滿，其...,1008,-1,-1,-1,-1,405
3,3,3,2018年度必追蹤台灣【最強IG美妝】達人帳號！各個試色狂人、超精準分析...購買前絕對必參考,2018/12/28,NICOLE HUANG,https://www.elle.com/tw/beauty/make-up/g256887...,以下這些台灣美妝INSTAGRAMERS各個都是不同風格的試色狂人，也非常用心經營各種彩妝教...,1542,-1,-1,-1,-1,-1
4,4,4,「新年擦紅唇怎麼選？顏色偏一點就顯牙黃...」擁有最完整26種紅色，植村秀超簡單選色教學公開,2018/12/28,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g256976...,紅色唇膏，是一個很玄妙的東西！偏橘或偏藍、上下深淺一個色階，讓你膚色變暗、牙齒變黃差很大！但...,539,-1,-1,572,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,187,187,秋冬裡的一把火紅唇風潮正在發燒！,2012/10/17,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2234/r...,全黑襯托赤色之美演出電影《SpringBreakers》的凡妮莎哈金斯（VanessaHud...,-1,-1,-1,263,-1,-1
188,188,188,紅唇魅力好萊塢女星展現復古風情,2012/06/29,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2652/o...,BlakeLively在LA出席電影首映會時，以金色側分復古捲髮現身，鮮紅唇色配上膚色幾何線...,-1,-1,-1,37,-1,-1
189,189,189,開賣立刻爆紅！YSL金管「#87柔瑰粉」色號必須收，比護唇膏還潤，媲美有顏色的唇膜啊...,2019/01/03,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g257317...,YSL全新情挑誘光水唇膏；暱稱金管，還未上市DCARD、小紅書討論度破表，金管配方升級，膏內...,-1,-1,-1,-1,66,-1
190,190,190,「狂掃30支送姊妹、家裡還有上百支唇膏！」標準唇膏控郭雪芙，最近卻情有獨鍾此色號。,2017/07/27,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g30416/...,過去的我在舞台上可能比較侷限於某一種單一形像，但其實我有很多面象還沒有機會展現給大家認識，尤...,-1,-1,-1,-1,426,-1


In [150]:
temp_all = 0
df["Indexes"] = df["內文"].str.find('唇妝')
df["Indexes1"] = df["內文"].str.find('唇彩')
df["Indexes2"] = df["內文"].str.find('口紅')
df["Indexes3"] = df["內文"].str.find('唇色')
df["Indexes4"] = df["內文"].str.find('護唇')
# df["Indexes5"] = df["內文"].str.find('眼線')

temp = df[df["Indexes"] != -1]
temp1 = df[df["Indexes1"] != -1]
temp2 = df[df["Indexes2"] != -1]
temp3 = df[df["Indexes3"] != -1]
temp4 = df[df["Indexes4"] != -1]
# temp5 = df[df["Indexes5"] != -1]

temp_all = pd.concat([temp,temp1,temp2,temp3,temp4], axis=0,ignore_index=True)
temp_all_1= temp_all.drop_duplicates(subset=None, keep='first')




In [39]:
df["Indexes1"] = df["標題"].str.find('影展')
temp = df[df["Indexes1"] != -1]
temp_num = temp['Unnamed: 0']
df1=df.drop(temp_num)
df1

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,標題,日期,作者,網址,內文,Indexes,Indexes1,Indexes2,Indexes3,Indexes4,Indexes5
0,0,0,0,2019最強開架唇膏評比！INTEGRATE、KATE、CLIO、ETUDE⋯不超過500元...,2019/10/12,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g258827...,雖然說專櫃唇膏總是讓人走心，但是近兩年不論是日系、韓系甚至於台灣本土開架彩妝品牌都不停推出適...,1680,-1,-1,52,-1,-1
1,1,1,1,草莓冰沙冰涼透潤紅美呆！「ARMANI#508、DIOR#976…」小紅書超人氣水潤唇膏推薦,2019/05/30,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g276247...,炎熱的夏季，除了靠穿搭營造換季成清涼配色之外，小紅書上也出現相當多適合夏季的水潤唇膏推薦，尤...,490,-1,-1,161,815,-1
2,2,2,2,「原來眉毛沒有型，氣質就很難突顯出來啊！」找對眉型臉整個小一號，超簡單圖解看完秒懂,2019/05/16,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g274888...,是不是覺得妝怎麼畫，就是少了一股空靈、氣質感？其實眉毛就是很重要的關鍵！不只是將空隙填滿，其...,1008,-1,-1,-1,-1,405
3,3,3,3,2018年度必追蹤台灣【最強IG美妝】達人帳號！各個試色狂人、超精準分析...購買前絕對必參考,2018/12/28,NICOLE HUANG,https://www.elle.com/tw/beauty/make-up/g256887...,以下這些台灣美妝INSTAGRAMERS各個都是不同風格的試色狂人，也非常用心經營各種彩妝教...,1542,-1,-1,-1,-1,-1
4,4,4,4,「新年擦紅唇怎麼選？顏色偏一點就顯牙黃...」擁有最完整26種紅色，植村秀超簡單選色教學公開,2018/12/28,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g256976...,紅色唇膏，是一個很玄妙的東西！偏橘或偏藍、上下深淺一個色階，讓你膚色變暗、牙齒變黃差很大！但...,539,-1,-1,572,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,289,289,289,「狂掃30支送姊妹、家裡還有上百支唇膏！」標準唇膏控郭雪芙，最近卻情有獨鍾此色號。,2017/07/27,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g30416/...,過去的我在舞台上可能比較侷限於某一種單一形像，但其實我有很多面象還沒有機會展現給大家認識，尤...,-1,-1,-1,-1,426,-1
278,290,290,290,YSL這5個裸粉色號很可以！擦出來是【粉嫩水潤】的美唇，資深櫃姐私底下都超愛,2017/07/27,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g30414/...,"（由左至右）1.YSL【情挑誘吻雙色蜜唇膏】N°2粉紅挑逗NT1,3502.YSL時尚印記絲...",-1,-1,-1,-1,476,-1
279,291,291,291,想要休閒隨興，又想要充滿時尚的大人味？！藍x銅玩色眼粧讓妳一次滿足！,2017/06/15,KAHLEN CHEN,https://www.elle.com/tw/beauty/make-up/g29655/...,溫柔的日落黃暈色調最適合展現你的女人味，亮銅色×暖灰色搭配組合的彩粧盤，任何膚色都能輕易上手...,-1,-1,-1,-1,257,161
280,292,292,292,【ELLE編輯美妝控真心話】我們的美妝收藏大公開！,2015/08/18,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2590/e...,原本以為自己最愛收集的是滿牆蠟燭，但卻發現蠟燭牆間穿插了幾塊好可愛的香皂，我ㄧ向對香香的、美...,-1,-1,-1,-1,610,-1


In [40]:
df1.to_json('./elle_teach/elle_lips_teach.json')
df1.to_excel('./elle_teach/elle_lips_teach.xlsx')

In [126]:
df.to_csv('./elle_teach/elle_lips_teach.csv')

In [4]:
df2 = df1.head(10)
df2

,內文,標題
0,高雄應該算是氣候最溫暖的城市了吧也因此，最近常騎車到處拍照的萍果，肌膚不但變黑，而且看起來就...,MINON保潤噴霧化妝水幫肌膚隨時補水
1,各位粉絲們好久不見啦想必大家對於我的新產品心得文一定引頸期盼很久啦對啦我好偷懶，最近都沒有什...,RealBarrier沛麗膚屏護保濕濃縮精華&屏護保濕潤澤水凝霜，長效持續48小時保濕，水份...
2,天氣正式邁入秋季，肌膚缺水的問題很容易開始出現了肌膚缺水、乾燥、乾癢的問題讓人很困擾，因為...,乾燥肌．孕期肌膚保養澳洲進口100%有機母嬰護膚品牌WillowbytheSea海洋微洛全效...
3,抗老是這幾年最常面對的課題，有時在想年輕時就懂使用抗老產品會不會現在老化問題就會少一些呢人們...,試用體驗秋冬必備高保濕又能對抗歲月痕跡的抗老保濕產品_﹝白慈﹞胎盤蛋白極抗齡精華原液
4,​又是從去年買，現在才用的眼影眼影雙十一即將到來，就趕緊讓大家看看CINDY&NICK家光影...,淘寶霧面眼影挑哪家呢Cindy&Nick光影交錯三色眼影
5,hihi我是hsuanhsuan不知道大家有沒有眼下細紋的困擾呢hsuanhsuan雖然不...,保養手帳紫色小熨斗，衣服要平整，肌膚也要抗皺LOREAL巴黎萊雅玻尿酸眼霜級撫紋精華霜紫熨斗
6,身為工作狂上司主管交代事項多到做不完常常因為工作壓力大肌膚容易暗沉、衰老抗老保養顯得非常重...,保養手帳頂級保養，抗老乳霜推薦雅詩蘭黛白金級黑鑽松露雙效奇蹟霜(松露霜
7,誰說平價粉底沒有好貨這篇就要來分享Novia近期用到CP值頗高的平價粉底。就是它來自Shil...,開架粉底液推薦2019CINQUAIN思珂3D光感精華粉底遮瑕度持妝度的表現都很優秀的高CP...
8,最愛的美妝驚喜盒又收到囉這個月的內容物也是超棒每個商品都很實用，剛好最近出國都能派上用場一直...,體驗開箱超值美妝保養驚喜盒ButyBo10月
9,效率保養專家BELDORA蓓朵娜時光青春露、HD智慧光感粉底乳霜展現我的自在美青春是最美好的...,效率保養專家BELDORA蓓朵娜時光青春露、HD智慧光感粉底乳霜展現我的自在美青春是最美好的時光


In [153]:
jieba.load_userdict(r'./userdict.txt')
#dfall = pd.real_excel(r'C:/Users/Big data/PycharmProjects/ourETL/userdict.txt')
wc_list = []
for n,i in enumerate (df):
    dd = jieba.lcut(i)
    for e in dd:
        if e =='眉毛' or e == '眼線':
            wc_list.append(n)
for b in wc_list:
    print(wc[b])

In [154]:
df

,標題,日期,作者,網址,內文,Indexes,Indexes1,Indexes2,Indexes3,Indexes4,Indexes5
編號,,,,,,,,,,,
0,火辣女神拐拐變身「阿凡達」，僅黏胸貼、穿丁字褲！揭露7小時變裝過程，同步公開健身菜單,2019/11/01,MICHELLE YANG,https://www.elle.com/tw/beauty/make-up/g296629...,過完萬聖節，不少名人皆在社群上分享自己當日的造型，小丑、安娜貝爾皆為熱門選項，而在網路上擁有...,-1.0,-1.0,-1.0,962.0,-1.0,-1.0
1,蘭蔻絕對完美唇膏爆紅色號務必抄下！《鬼怪》金高恩#264、秀智專屬色S#360...每支都美到炸,2019/10/30,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g27922/...,這部人氣王《鬼怪》荷爾蒙裂屏的孔侑、單眼皮美女金高恩、高挑有型的李棟旭…無一不吸引著妳，也覺...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,優雅的大人如何重拾玩心？今年用全新蛻變登場的LUNASOL霓光系列征服五光十色的派對季節,2019/10/28,CHIA CHIU,https://www.elle.com/tw/beauty/make-up/a296039...,喜歡畫大地色系妝容的人，一定都用過有「大地色女王」之美名的LUNASOL，大地色系如此優雅迷...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,【老闆不說的事】業界最美公關！資生堂傳媒執行長張凡恬：「帶人要帶心！員工有怨氣先帶去喝一杯」,2019/10/24,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g295693...,談到最愛彩妝、保養，所有台灣女生的化妝桌上絕對少不了SHISEIDO資生堂集團的產品！蘿拉蜜...,-1.0,999.0,-1.0,1187.0,-1.0,-1.0
4,推薦給不張狂、低調優雅的妳！爆紅【可可裸棕】唇膏，6款人氣色號完整試色,2019/10/23,ROYAL HUANG,https://www.elle.com/tw/beauty/make-up/g244652...,唇彩大勢繼乾燥玫瑰色系、鐵鏽紅色系後，爆紅的「南瓜土橘色」重磅登場！混合暖色調的土色，宛如一...,347.0,198.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...
1161,10位最美金球獎女星征服紅毯全紀錄,2012/01/18,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2596/1...,紅唇搖滾全場-安潔莉娜裘莉安潔莉娜裘莉(AngelinaJolie)與黛安娜艾格倫(Dian...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1162,2012好萊塢美女預測新亮相,2012/01/12,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2595/2...,FelicityJones大家可能對這位英國女演員不太熟悉(演過TheWorstWitch和...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1163,好想咬一口！打造女星們的水蜜桃美肌,2012/01/10,ELLE TAIWAN EDITOR,https://www.elle.com/tw/beauty/make-up/g2594/c...,亞曼達賽佛瑞(AmandaSeyfried)亞曼達賽佛瑞(AmandaSeyfried)天生...,-1.0,61.0,-1.0,169.0,291.0,-1.0


In [148]:
temp_all_1.to_json('./elle_teach/elle_face_teach.json')
temp_all_1.to_excel('./elle_teach/elle_face_teach.xlsx')

In [5]:
!pip install dfply

In [ ]:
fk